### FINAL PROJECT - JEN, THEA, SARATH 
### --- DATING ANALYSIS ---
### PREDICT WHETHER DATERS WANT TO MEET AGAIN

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pltt
import seaborn as sns
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score
scorer = make_scorer(fbeta_score, beta=1.5)

# Some warnings tend to pop up during grid search
import warnings
warnings.filterwarnings('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('Data/Speed_Dating_Data.csv', encoding='latin-1', engine="python")

print(data.shape[0], 'observations and', data.shape[1], 'features')

8378 observations and 195 features


In [3]:
# What kind of data are we dealing with?
data.dtypes.value_counts()

float64    174
int64       13
object       8
dtype: int64

In [4]:
# Men (1) and women (0)
data.groupby('iid').mean().gender.value_counts()

1.0    277
0.0    274
Name: gender, dtype: int64

In [5]:
# What percentage of pairs resulted in a match?
data['match'].value_counts(normalize=True)

0    0.835283
1    0.164717
Name: match, dtype: float64

In [6]:
data.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [7]:
# Waves 6-11 rated importance, others allocated 100 points
points = (data.wave != 6 ) &\
    (data.wave != 7 ) &\
    (data.wave != 8 ) &\
    (data.wave != 9 ) &\
    (data.wave != 10 ) &\
    (data.wave != 11 )

data = data[points]

In [8]:
match_corrs = data.select_dtypes(include=[np.number])\
.corrwith(data.match)\
.sort_values(ascending=False)

match_corrs = match_corrs[match_corrs > .25].index

data = data.dropna(subset=['id', 'pid'], axis=0)

for i in match_corrs[1:]:
    del data[i]

# Other columns that are too predictive
#del data['int_corr']
del data['them_cal']
del data['you_call']

del data['field'] # redundant

match_corrs

Index(['match', 'dec_o', 'dec', 'like_o', 'like', 'fun_o', 'fun', 'shar_o',
       'shar', 'attr_o', 'attr', 'prob_o', 'prob'],
      dtype='object')

In [9]:
def fix_numeric(col, df=data, median=False):
    try:
        df[col] = df[col].str.replace(',', '')
    except:
        print('Column is not a string!')
    
    df[col] = df[col].astype(float)
    
    if median:
        df[col] = df[col].fillna(data.fillna.median())
    
    else:
        df[col] = df[col].fillna(-100)
    
    return df

for i in 'zipcode mn_sat tuition income'.split():
    data = fix_numeric(i)

In [10]:
for i in data.columns:
    if data[i].dtype == "O":
        data[i] = data[i].str.lower()
        print(i, end=', ')

undergra, from, career, 

In [11]:
from sklearn.dummy import DummyClassifier

data_benchmark = pd.get_dummies(data)
data_benchmark.fillna(data_benchmark.median(), inplace=True)

xbench = data_benchmark.drop('match', axis=1)
ybench = data_benchmark['match']

dum = DummyClassifier(strategy='constant', constant=1)

cvs = cross_val_score(dum, xbench, ybench, scoring=scorer, cv=5)

benchmark_score = cvs.mean()

print('Naive Benchmark:', benchmark_score)

Naive Benchmark: 0.39567670124395443


In [12]:
data['from_new_england'] = 0
for i in 'massachu connect rhode vermont vt hampsh maine boston cambridge'.split():
    data['from_new_england'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                        data['from_new_england'])
    
data['from_china'] = 0
for i in 'china beijing shanghai hong taiwan'.split():
    data['from_china'] = np.where((data['from'].str.contains(i)) |\
                                  (data['undergra'].str.contains(i)), 1,
                                  data['from_china'])

data['from_india'] = 0
for i in 'india delhi bangalore'.split():
    data['from_india'] = np.where((data['from'].str.contains(i)) |\
                                  (data['undergra'].str.contains(i)), 1,
                                  data['from_india'])
    
data['from_europe'] = 0
for i in 'europe germany italy france spain poland portugal netherlands holland sweden switz greece belgium paris rome'.split():
    data['from_europe'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                   data['from_europe'])

data['from_uk'] = 0
for i in 'london england uk britain scotland ireland kingdom oxford'.split():
    data['from_uk'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                               data['from_uk'])

data['from_ny'] = 0
for i in ['new york', 'ny']:
    data['from_ny'] = np.where((data['from'].str.contains(i)) |\
                               (data['undergra'].str.contains(i)), 1,
                               data['from_ny'])

data['from_nj'] = 0
for i in ['new jersey', 'nj']:
    data['from_nj'] = np.where((data['from'].str.contains(i)) |\
                               (data['undergra'].str.contains(i)), 1,
                               data['from_nj'])

data['from_california'] = 0
for i in 'cali diego francisco jose davis sacramento oakland clara angeles ucla stanford berkeley alto torrance'.split():
    data['from_california'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                       data['from_california'])
    
data['from_texas'] = 0
for i in 'texas tx dallas austin houston dfw antonio'.split():
    data['from_texas'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                  data['from_texas'])
    

In [13]:
data['career_education'] = 0
for i in 'professor teach academ educ'.split():
    data['career_education'] = np.where(data['career'].str.contains(i),
                                        1, data['career_education'])

data['career_law'] = 0
for i in 'law attorney legal defender counsel'.split():
    data['career_law'] = np.where(data['career'].str.contains(i),
                                  1, data['career_law'])

data['career_medicine'] = 0
for i in 'doctor dr physician md medical m.d. cardio dentist surg'.split():
    data['career_medicine'] = np.where(data['career'].str.contains(i),
                                       1, data['career_medicine'])

data['career_business'] = 0
for i in 'business mba m.b.a. consult manage ceo c.e.o. entre finance venture market strategy invest bank equity'.split():
    data['career_business'] = np.where(data['career'].str.contains(i),
                                       1, data['career_business'])
    
data['career_science'] = 0
for i in 'sci research biolo chemi'.split():
    data['career_science'] = np.where(data['career'].str.contains(i),
                                      1, data['career_science'])

data['career_gov'] = 0
for i in 'gov diplo poli'.split():
    data['career_gov'] = np.where(data['career'].str.contains(i),
                                  1, data['career_gov'])


In [14]:
data['school_columbia'] = 0
for i in 'columbia'.split():
    data['school_columbia'] = np.where(data['undergra'].str.contains(i),
                                       1, data['school_columbia'])

# Ivy league and other prestigious schools
data['school_ivy'] = 0
for i in 'dartmouth cornell princeton penn yale brown harvard stanford mit berkeley oxford'.split():
    data['school_ivy'] = np.where(data['undergra'].str.contains(i),
                                  1, data['school_ivy'])

In [15]:
# Dummify
data = pd.get_dummies(data, prefix='dummy')

# Some Stackoverflow code to fix duplicate column names
cols = pd.Series(data.columns)

for dup in data.columns.get_duplicates():
    cols[data.columns.get_loc(dup)] =\
    [dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(data.columns.get_loc(dup).sum())]

data.columns = cols

data.shape

(5761, 821)

In [16]:
# Just a sample of how much missing data we have.
data.isnull().sum().sort_values(ascending=False)[:10]

num_in_3    5278
numdat_3    5067
shar4_3     4271
attr7_3     4271
sinc7_3     4271
intel7_3    4271
fun7_3      4271
amb7_3      4271
attr4_3     4271
sinc4_3     4271
dtype: int64

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
# Get null columns
null_cols = data.isnull().sum()
null_cols = list(null_cols[null_cols != 0].index)

for i in null_cols:
    print(i, 'has', data[i].isnull().sum(), 'missing values')
    x = data.fillna(data[i].mean()).drop(['match', 'id'], axis=1)
    y = x.pop(i)
    
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=.2)
    reg = KNeighborsRegressor()
    reg.fit(xtrain, ytrain)      
    pred = reg.predict(xtest)
    r2 = r2_score(ytest, pred)
    
    # If we can reasonably predict these values, do so
    if r2 > .2:
        print('regressing', i, 'with an r2 of', round(r2, 3))
        data['predicted'] = reg.predict(data.fillna(data[i].median()).drop([i, 'match', 'id'], axis=1))
        data[i] = np.where(data[i].isnull(), data['predicted'], data[i])
        del data['predicted']
    
    # Otherwise, just take the median
    else:
        print('averaging', i)
        data[i] = data[i].fillna(data[i].median())
    print()

positin1 has 1836 missing values
regressing positin1 with an r2 of 0.885

int_corr has 148 missing values
averaging int_corr

age_o has 89 missing values
averaging age_o

race_o has 58 missing values
averaging race_o

pf_o_att has 74 missing values
regressing pf_o_att with an r2 of 0.261

pf_o_sin has 74 missing values
averaging pf_o_sin

pf_o_int has 74 missing values
averaging pf_o_int

pf_o_fun has 83 missing values
averaging pf_o_fun

pf_o_amb has 92 missing values
averaging pf_o_amb

pf_o_sha has 114 missing values
averaging pf_o_sha

sinc_o has 232 missing values
averaging sinc_o

intel_o has 242 missing values
averaging intel_o

amb_o has 505 missing values
averaging amb_o

met_o has 314 missing values
averaging met_o

age has 89 missing values
regressing age with an r2 of 0.987

field_cd has 77 missing values
regressing field_cd with an r2 of 0.999

race has 58 missing values
regressing race with an r2 of 0.999

imprace has 74 missing values
regressing imprace with an r2 of 0.9

regressing intel5_2 with an r2 of 0.999

fun5_2 has 2387 missing values
regressing fun5_2 with an r2 of 0.997

amb5_2 has 2387 missing values
regressing amb5_2 with an r2 of 0.994

date_3 has 3261 missing values
regressing date_3 with an r2 of 0.997

numdat_3 has 5067 missing values
regressing numdat_3 with an r2 of 0.997

num_in_3 has 5278 missing values
regressing num_in_3 with an r2 of 1.0

attr1_3 has 3261 missing values
regressing attr1_3 with an r2 of 0.99

sinc1_3 has 3261 missing values
regressing sinc1_3 with an r2 of 1.0

intel1_3 has 3261 missing values
regressing intel1_3 with an r2 of 0.998

fun1_3 has 3261 missing values
regressing fun1_3 with an r2 of 1.0

amb1_3 has 3261 missing values
regressing amb1_3 with an r2 of 0.994

shar1_3 has 3261 missing values
regressing shar1_3 with an r2 of 0.999

attr7_3 has 4271 missing values
regressing attr7_3 with an r2 of 0.995

sinc7_3 has 4271 missing values
regressing sinc7_3 with an r2 of 0.999

intel7_3 has 4271 missing values
r

In [ ]:
def get_partner_data(pid, col):
    '''Looks up the person's partner and adds their data
    as new features. If the partner ID doesn't exist,
    returns a -1.'''
    try:
        partner = data[data['iid'] == pid].head(1)[col].iloc[0]
        if partner:
            return partner
        else:
            return -1
    except:
        return -1

In [ ]:
# Income (where income data is available, take the log difference)
data['partner_income'] = data['pid'].apply(get_partner_data, col='income')
data['income_difference'] = np.where((data.partner_income == -1) |\
                                     (data.income == -1),
                                     -1, np.log1p(np.abs(data.income - data.partner_income)))

# Age
data['age'].fillna(data['age'].median(), inplace=True)
data['age_o'].fillna(data['age_o'].median(), inplace=True)
data['age_difference'] = data['age'] - data['age_o']

# Attractiveness
data['partner_attractiveness'] = data['pid'].apply(get_partner_data, col='attr3_1')
data['attr_expectations'] = (10*data['attr1_1']/data['attr1_1'].max()) -\
                                     data['partner_attractiveness']
data['partner_attractiveness_2'] = data['pid'].apply(get_partner_data, col='attr5_1')
data['attr_expectations_2'] = (10*data['attr1_1']/data['attr1_1'].max()) -\
                                       data['partner_attractiveness_2']

# Intelligence
data['partner_intelligence'] = data['pid'].apply(get_partner_data, col='intel3_1')
data['intel_expectations'] = (10*data['intel1_1']/data['intel1_1'].max()) -\
                                     data['partner_intelligence']
data['partner_intelligence_2'] = data['pid'].apply(get_partner_data, col='intel5_1')
data['intel_expectations_2'] = (10*data['intel1_1']/data['intel1_1'].max()) -\
                                       data['partner_intelligence_2']

# Fun
data['partner_fun'] = data['pid'].apply(get_partner_data, col='fun3_1')
data['fun_expectations'] = (10*data['fun1_1']/data['fun1_1'].max()) -\
                                     data['partner_fun']
data['partner_fun_2'] = data['pid'].apply(get_partner_data, col='fun5_1')
data['fun_expectations_2'] = (10*data['fun1_1']/data['fun1_1'].max()) -\
                                       data['partner_fun_2']

# Ambitious
data['partner_ambition'] = data['pid'].apply(get_partner_data, col='amb3_1')
data['amb_expectations'] = (10*data['amb1_1']/data['amb1_1'].max()) -\
                                     data['partner_ambition']
data['partner_ambition_2'] = data['pid'].apply(get_partner_data, col='amb5_1')
data['amb_expectations_2'] = (10*data['amb1_1']/data['amb1_1'].max()) -\
                                       data['partner_ambition_2']

# Sincerity
data['partner_sincerity'] = data['pid'].apply(get_partner_data, col='sinc3_1')
data['sinc_expectations'] = (10*data['sinc1_1']/data['sinc1_1'].max()) -\
                                     data['partner_sincerity']
data['partner_sincerity_2'] = data['pid'].apply(get_partner_data, col='sinc5_1')
data['sinc_expectations_2'] = (10*data['sinc1_1']/data['sinc1_1'].max()) -\
                                       data['partner_sincerity_2']

In [ ]:
#Measuring Expectations
# Measuring expectations one more way

data['attr_above_avg'] = np.where(data['partner_attractiveness'] > data['partner_attractiveness'].mean(),
                                  1, 0)
data['attr_imp_above_avg']= np.where(data['attr1_1'] > data['attr1_1'].mean(),
                                     1, 0)
data['attr_match'] = np.where(data['attr_above_avg'] >= data['attr_imp_above_avg'],
                             1, 0)

data['sinc_above_avg'] = np.where(data['partner_sincerity'] > data['partner_sincerity'].mean(),
                                  1, 0)
data['sinc_imp_above_avg']= np.where(data['sinc1_1'] > data['sinc1_1'].mean(),
                                     1, 0)
data['sinc_match'] = np.where(data['sinc_above_avg'] >= data['sinc_imp_above_avg'],
                             1, 0)

data['int_above_avg'] = np.where(data['partner_intelligence'] > data['partner_intelligence'].mean(),
                                  1, 0)
data['int_imp_above_avg']= np.where(data['intel1_1'] > data['intel1_1'].mean(),
                                     1, 0)
data['int_match'] = np.where(data['int_above_avg'] >= data['int_imp_above_avg'],
                             1, 0)

data['fun_above_avg'] = np.where(data['partner_fun'] > data['partner_fun'].mean(),
                                  1, 0)
data['fun_imp_above_avg']= np.where(data['fun1_1'] > data['fun1_1'].mean(),
                                     1, 0)
data['fun_match'] = np.where(data['fun_above_avg'] >= data['fun_imp_above_avg'],
                             1, 0)

In [ ]:
# Ambition (recall we're doing this one differently)

# Temporarily replace missing values with the median
data['income_temp'] = np.where(data['income'] == -100, data['income'].median(), data['income'])

data['income_above_avg']= np.where(data['income_temp'] > data['income_temp'].median(), 1, 0)
data['partner_income_above_avg'] = data['pid'].apply(get_partner_data, col='income_above_avg')

data['amb_imp_above_avg'] = np.where(data['amb1_1'] > data['amb1_1'].mean(), 1, 0)

data['amb_match'] = np.where(data['partner_income_above_avg'] >= data['amb_imp_above_avg'],
                             1, 0)
del data['income_temp']

In [ ]:
cols = data.columns

for i in cols:
    if 'from_' in i:
        data['partner_loc'] = data['pid'].apply(get_partner_data, col=i)
        data['both_' + i] = np.where((data[i] == 1) & (data['partner_loc'] == 1), 1, 0)
        del data['partner_loc']
        
for i in cols:
    if 'school_' in i:
        data['partner_edu'] = data['pid'].apply(get_partner_data, col=i)
        data['both_' + i] = np.where((data[i] == 1) & (data['partner_edu'] == 1), 1, 0)
        del data['partner_edu']
        
for i in cols:
    if 'career_' in i:
        data['partner_car'] = data['pid'].apply(get_partner_data, col=i)
        data['both_' + i] = np.where((data[i] == 1) & (data['partner_car'] == 1), 1, 0)
        del data['partner_car']

In [ ]:
#Hobbies
hobbies = ['go_out', 'sports', 'tvsports', 'exercise', 'dining', 'museums',
 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater',
 'movies', 'concerts', 'music', 'shopping', 'yoga']

for i in hobbies:
    data['partner_' + i] = data['pid'].apply(get_partner_data, col=i)
    data[i + '_in_common'] = np.where((data[i] > 7) & (data['partner_' + i] > 7), 1, 0)
    del data['partner_' + i]

In [ ]:
#Remove dummies when one person has 1

#****Remove # to delete data after running first time
counts =\
data.groupby('iid').mean().sum(axis=0).sort_values()

deleted = []

for i in counts[counts < 2].index:
    if 'dummy_' in i:
        deleted.append(i)
       # del data[i]

# Fix anything that got counted twice
for i in data.columns:
    if 'dummy_' in i:
        data[i] = np.where(data[i] >= 1, 1, 0)

In [ ]:
# Nearly 500 features removed. Not bad!
print('Deleted:', len(deleted))
print('Data shape:', data.shape)

In [ ]:
#Processing Zipcode Data
data['zipcode'] = data['zipcode'].astype(str)
data['zip_5'] = (data['zipcode'].str[:5]).astype(float)
data['zip_4'] = (data['zipcode'].str[:4]).astype(float)
data['zip_3'] = (data['zipcode'].str[:3]).astype(float)
data['zip_2'] = (data['zipcode'].str[:2]).astype(float)
del data['zipcode']

data['partner_zip'] = data['pid'].apply(get_partner_data, col='zipcode')
data['partner_zip'] = data['partner_zip'].astype(str)
data['partner_zip_5'] = (data['partner_zip'].str[:5]).astype(float)
data['partner_zip_4'] = (data['partner_zip'].str[:4]).astype(float)
data['partner_zip_3'] = (data['partner_zip'].str[:3]).astype(float)
data['partner_zip_2'] = (data['partner_zip'].str[:2]).astype(float)
del data['partner_zip']

In [ ]:
#Checking correlations
cols = data.columns


for i in cols:
    for j in cols:
        if i > j:
            corr = np.corrcoef(data[i], data[j])[0,0]
            if corr > .90:
                print(i, j)

In [ ]:
print('Data shape:', data.shape)

In [ ]:
# Checkpoint
data.to_csv('Speed Dating Data Processed.csv')

In [ ]:
###Visualizations
sns.lmplot("attr1_1", "amb1_1",
           data=data, hue='gender', fit_reg=False,
           x_jitter=5, y_jitter=5,
           palette=['magenta', 'blue'],
           scatter_kws={'alpha':.05})
plt.title('Importance of attractiveness vs. ambition in men and women')

In [ ]:
sns.kdeplot(data[data.gender == 1].age,
            bw=1.5, shade=True, color='blue', label='men')
sns.kdeplot(data[data.gender == 0].age,
            bw=1.5, shade=True, color='magenta', label='women')
plt.title('Age of daters')
plt.xlabel('Age')

In [ ]:
sns.kdeplot(data[(data.gender == 1) & (data.income > 0)].income,
            bw=15000, shade=True, color='blue', label='men')
sns.kdeplot(data[(data.gender == 0) & (data.income > 0)].income,
            bw=15000, shade=True, color='magenta', label='women')
plt.title('Estimated income of daters')

plt.xlim(0,150000)
plt.xlabel('Zip code median income')

In [ ]:
#Gridsearch
from sklearn.model_selection import train_test_split, GridSearchCV

x = data.drop('match', axis=1)
y = data['match']

# Fixing a csv import bug
for i in x.columns:
    if 'Unnamed' in i:
        del x[i]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=.2)

xtrain.shape

In [ ]:
#Using K-Neighbor Classifier
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
params = {'n_neighbors': np.arange(1,9,2)}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
params = {'penalty': ['l1', 'l2'],
         'C': [.1, 1, 10, 100]}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
params = {'max_depth': list(np.arange(2,13)) + [None],
         'min_samples_leaf': [1,2,3,4,5,6,7,10]}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
params = {'max_depth': list(np.arange(3,10)) + [None],
         'min_samples_leaf': [1,2,3],
         'n_estimators': [10,100]}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
# Some Stackoverflow code to fix duplicate column names AGAIN
cols = pd.Series(data.columns)

for dup in data.columns.get_duplicates():
    cols[data.columns.get_loc(dup)] =\
    [dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(data.columns.get_loc(dup).sum())]

data.columns = cols

data.shape

In [ ]:
from xgboost import XGBClassifier


clf = XGBClassifier()
params = {'max_depth': list(np.arange(3,10)) + [100],
         'base_score': np.arange(.42,.52,.01),
         'n_estimators': [100]}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()
params = {'max_depth': [9],
         'base_score': np.arange(.40,.45,.007),
         'n_estimators': [100]}
grid = GridSearchCV(clf, param_grid=params, cv=4, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=4, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
#Dimensionality Reduction & Grid Search

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV, train_test_split

pl = Pipeline([
    ('reduce_dim', SelectPercentile()),
    ('classify', XGBClassifier())
])

# Recall that I've run this notebook multiple times
# and did a MUCH more exhaustive search previously.
PERC_OPTIONS = np.arange(44,58,2) # It consistently settles on a value around 50-55
EST_OPTIONS = [400] # Best setting I found; we'll only use this to save time
DEPTH_OPTIONS = [6,7,8,9]

param_grid = [
    {
        'reduce_dim__percentile': PERC_OPTIONS,
        'classify__n_estimators': EST_OPTIONS,
        'classify__max_depth': DEPTH_OPTIONS,
    },
]

grid =\
GridSearchCV(pl, cv=3, n_jobs=1, param_grid=param_grid, scoring=scorer)\
.fit(xtrain, ytrain)

model = grid.best_estimator_

In [ ]:
model.named_steps['classify']

In [ ]:
model.named_steps['reduce_dim']

In [ ]:
cv = cross_val_score(model, xtrain, ytrain, cv=4, scoring=scorer)

print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class MatchMakingClassifier(BaseEstimator, ClassifierMixin):
    
    def __init__(self, estimator, threshold=50):
        self.estimator = estimator
        self.threshold = threshold

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    
    def predict(self, X):
        pred = self.estimator.predict_proba(X)[:,1]
        pred = np.where(pred > np.percentile(pred, self.threshold), 1, 0)
        return pred

In [ ]:
clf = MatchMakingClassifier(model)
params = {'threshold': [48,48.5,49]}
grid = GridSearchCV(clf, param_grid=params, cv=5, scoring=scorer).fit(xtrain, ytrain)
clf = grid.best_estimator_

cv = cross_val_score(clf, xtrain, ytrain, cv=5, scoring=scorer)

print(clf, '\n')
print('Mean score:', cv.mean())
print('Std Dev:   ', cv.std())

In [ ]:
pred = clf.predict(xtest)

fbeta_score(ytest, pred, beta=1.5)

In [ ]:
#Sensitivity Analysis

In [ ]:
from random import choice

def sensitivity_analysis(x=x, clf=clf, cols=10, print_scrambled=False):
    '''
    Originally I tried shuffling and scrambling the
    data. This takes a LONG time, so I eventually
    settled on simply randoming it.
    '''
    cv_scores = []
    
    # Average over 5 times
    for i in range(5):
        x_sens = x.copy()
        scrambled = []
        while len(scrambled) < cols:
            col_to_scramble = choice(x_sens.columns)
            scrambled.append(col_to_scramble)
            x_sens[col_to_scramble] = np.random.randn(len(x_sens))
        if print_scrambled:
            print('Scrambed:', scrambled)
        cv_scores.append(cross_val_score(clf, x_sens, y, cv=4, scoring=scorer).mean())
        
    print('Average cross val score ({} columns scrambled):'.format(cols), np.mean(cv_scores))

In [ ]:
sensitivity_analysis(cols=10)

In [ ]:
sensitivity_analysis(cols=20)

In [ ]:
sensitivity_analysis(cols=50)

In [ ]:
sensitivity_analysis(cols=75)

In [ ]:
sensitivity_analysis(cols=100)

In [ ]:
sensitivity_analysis(cols=200)

In [ ]:
sensitivity_analysis(cols=400)

In [ ]:
#Cross Validation
cv = cross_val_score(clf, x, y, cv=4, scoring=scorer)

print('Mean f-beta: {0:.4f} (Std dev: {1:.4f})'.format(cv.mean(),  cv.std()))

In [ ]:
(cv.mean() - benchmark_score)/benchmark_score

In [ ]:
# Code from the sklearn documentation

from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(x.values, y.values):
    probas_ = model.fit(x[train], y[train]).predict_proba(x[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Luck', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# For simplicity's sake, we'll train a new XGBoost classifier
xg = XGBClassifier(max_depth=12, n_estimators=400).fit(x,y)
xg = grid.best_estimator_

fig, ax = plt.subplots(figsize=(12, len(x.columns)//2))

importances = pd.Series(xg.feature_importances_,
                        index=x.columns).sort_values(ascending=True)

importances.plot.barh()